<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [6]:
import pandas as pd
import numpy as np
x1 = [0, 0, 1, 0, 1, 1, 0]
x2 = [0, 1, 0, 1, 0, 1, 0]
x3 = [1, 1, 1, 0, 0, 1, 0]
y = [0, 1, 1, 1, 1, 0, 0]
df = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'y': y})
df

,x1,x2,x3,y
0,0,0,1,0
1,0,1,1,1
2,1,0,1,1
3,0,1,0,1
4,1,0,0,1
5,1,1,1,0
6,0,0,0,0


In [7]:
X = df[['x1','x2','x3']].values.astype(float)
y = df[['y']].values.astype(float)

In [8]:
X

array([[0., 0., 1.],
       [0., 1., 1.],
       [1., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 1., 1.],
       [0., 0., 0.]])

In [15]:
y

array([[0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.]])

In [9]:
class NeuralNetwork:
    
    def __init__(self):
        # Set up Architecture of Neural Network
        self.inputs = 3
        self.hiddenNodes =  4
        self.outputNodes = 1

        # Initial Weights
        # 2x3 Matrix Array for the First Layer
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes)
       
        # 3x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1-s)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward pass.
        """
        
        # Weighted sum of inputs and hidden layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activation of the weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weighted sum of hidden layer to output layer
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final Activation of Output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    def backward(self, X, y, o):
        
        self.o_error = y - o #error in output
        
        # Size of adjustment from hidden to output
        self.o_delta = self.o_error * self.sigmoidPrime(o) #apply derivative of sigmoid to error
        
        #z2 error: how much our hidden layer to input weights were off
        self.z2_error = self.o_delta.dot(self.weights2.T)
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        # Adjustment hidden => output weights
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)
        
        #Adjustment input => hidden weights
        self.weights1 += X.T.dot(self.z2_delta)
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [13]:
nn = NeuralNetwork()

for i in range(5000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 200 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        #print('Actual Output: \n', y)
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X,y)
#     if np.array_equal(np.rint(nn.feed_forward(X)), y):
#         print(f'Final Predicted Output: \n{ np.rint(nn.feed_forward(X))}')
#         print(f'Epoch: {i+1}')
#         break

+---------EPOCH 1---------+
Predicted Output: 
 [[0.21262879]
 [0.25431188]
 [0.33460532]
 [0.38360835]
 [0.47587699]
 [0.41168755]
 [0.35213694]]
Loss: 
 0.2845917857213064
+---------EPOCH 2---------+
Predicted Output: 
 [[0.30173972]
 [0.38979584]
 [0.45003058]
 [0.51824923]
 [0.58070676]
 [0.55969798]
 [0.44239271]]
Loss: 
 0.24038944435802764
+---------EPOCH 3---------+
Predicted Output: 
 [[0.36254332]
 [0.48202877]
 [0.5244421 ]
 [0.59791966]
 [0.64091118]
 [0.64614396]
 [0.49685535]]
Loss: 
 0.22583825853826658
+---------EPOCH 4---------+
Predicted Output: 
 [[0.38951327]
 [0.52511593]
 [0.56041333]
 [0.63310131]
 [0.66888268]
 [0.6853838 ]
 [0.51965512]]
Loss: 
 0.2220739456154525
+---------EPOCH 5---------+
Predicted Output: 
 [[0.39891292]
 [0.54431448]
 [0.57822374]
 [0.64873304]
 [0.68285722]
 [0.70449882]
 [0.52747427]]
Loss: 
 0.22045596332814135
+---------EPOCH 200---------+
Predicted Output: 
 [[0.09085782]
 [0.73033708]
 [0.6677216 ]
 [0.89531081]
 [0.80466804]
 [0.707

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [25]:
from tensorflow import keras

In [16]:
from keras.datasets import fashion_mnist
((X_train, y_train), (X_test, y_test)) = fashion_mnist.load_data()

Using TensorFlow backend.


In [17]:
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [18]:
# Reshape the data into single vectors
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)

In [19]:
# X Variable Types
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [27]:
# normalize to keep the gradients manageable
X_train = X_train/255
X_test = X_test/255

In [23]:
X_train.min()

0.0

In [28]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

In [26]:
# Correct encoding of y
# What softmax expects = [0,0,0,0,0,1,0,0,0,0]

y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

In [ ]:
class NNDigit(object:
    
    def __init__(self, epochs=20, batch_size=None, learning_rate=0.1):
        # Set up Architecture of Neural Network
        self.lr = learning_rate
        self.epochs = epochs
        self.inputs = 784
        self.hiddenNodes =  10
        self.outputNodes = 10

        # Initial hidden layer
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes)
       
        # 3x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
        self.bias2 = np.zeros((10, 1))
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1-s)
    
    def compute_loss(Y, Y-hat):
        m = Y.shape[1]
        L = -(1./m) * (np.sum( np.multiply(np.log(Y_hat),Y)) + np.sum(np.multiply(np.log(1-Y_hat),(1-Y))))
        return L

## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?